# Using your own object detector for detection images

This notebook is based on the [official Tensorflow Object Detection demo](https://github.com/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb) and only contains some slight changes. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

# Imports

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')

## Env setup

In [2]:
# This is needed to display the images.
%matplotlib inline

## Object detection imports
Here are the imports from the object detection module.

In [3]:
from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_FROZEN_GRAPH` to point to a new .pb file.  

In [4]:
MODEL_NAME = 'inference_graph'
PATH_TO_FROZEN_GRAPH = 'trained-inference-graphs/output_inference_graph_v1.pb/frozen_inference_graph.pb'
PATH_TO_LABELS = 'annotations/label_map.pbtxt'

## Load a (frozen) Tensorflow model into memory.

In [5]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [6]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

# Detection

In [7]:
def run_inference_for_single_image(image, graph):
    if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
    image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

    # Run inference
    output_dict = sess.run(tensor_dict,
                            feed_dict={image_tensor: np.expand_dims(image, 0)})

    # all outputs are float32 numpy arrays, so convert types as appropriate
    output_dict['num_detections'] = int(output_dict['num_detections'][0])
    output_dict['detection_classes'] = output_dict[
        'detection_classes'][0].astype(np.uint8)
    output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
    output_dict['detection_scores'] = output_dict['detection_scores'][0]
    if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict

In [23]:
# def multiply(x):
#     return x * 800


# import cv2
# cap = cv2.VideoCapture("test2.mp4")
# try:
#     with detection_graph.as_default():
#         with tf.Session() as sess:
#                 # Get handles to input and output tensors
#                 ops = tf.get_default_graph().get_operations()
#                 all_tensor_names = {output.name for op in ops for output in op.outputs}
#                 tensor_dict = {}
#                 for key in [
#                   'num_detections', 'detection_boxes', 'detection_scores',
#                   'detection_classes', 'detection_masks'
#                 ]:
#                     tensor_name = key + ':0'
#                     if tensor_name in all_tensor_names:
#                         tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
#                       tensor_name)

#                 while True:
#                     ret, image_np = cap.read()
#                     # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
#                     image_np_expanded = np.expand_dims(image_np, axis=0)
#                     # Actual detection.
#                     output_dict = run_inference_for_single_image(image_np, detection_graph)
#                     coordinates = np.asarray(output_dict['detection_boxes'])*800
#                     print(np.asarray(coordinates))
                    
#                     cv2.rectangle(image_np, (int(coordinates[-1][1]), int(coordinates[-1][0])), (int(coordinates[-1][3]), int(coordinates[-1][2])), (255,0,0), 2)
#                     # Visualization of the results of a detection.
#                     vis_util.visualize_boxes_and_labels_on_image_array(
#                         image_np,
#                         output_dict['detection_boxes'],
#                         output_dict['detection_classes'],
#                         output_dict['detection_scores'],
#                         category_index,
#                         instance_masks=output_dict.get('detection_masks'),
#                         use_normalized_coordinates=True,
#                         line_thickness=8)
#                     cv2.imshow('object_detection', cv2.resize(image_np, (800, 800)))
#                     if cv2.waitKey(25) & 0xFF == ord('q'):
#                         cap.release()
#                         cv2.destroyAllWindows()
#                         break
# except Exception as e:
#     print(e)
#     cap.release()

[[ 74.37456  192.6086   577.61115  378.03674 ]
 [550.3731   105.84693  800.       302.61246 ]
 [  0.       477.09918  554.591    800.      ]
 [  0.       335.48788  316.731    800.      ]
 [285.6921     0.       800.       286.19485 ]
 [403.43634  567.0982   800.       800.      ]
 [413.23672  460.80228  800.       800.      ]
 [415.1269   258.03458  800.       574.46814 ]
 [307.46088  363.36536  599.6684   658.45557 ]
 [629.0192   192.26633  800.       513.196   ]
 [330.18497  395.41873  800.       738.77203 ]
 [514.3258   196.59383  800.       383.86954 ]
 [  0.       371.71103  373.67557  800.      ]
 [  0.         0.       128.98935  566.8837  ]
 [242.45288  341.47495  659.6622   800.      ]
 [592.52026  427.42444  800.       800.      ]
 [473.3634   311.40784  787.53845  800.      ]
 [454.7903   504.27414  800.       702.69244 ]
 [  0.         0.       408.50867  360.79965 ]
 [527.2754   387.90845  800.       641.6728  ]
 [450.37814  168.3025   800.       660.5526  ]
 [  0.       

[[ 78.491684 190.93852  589.4601   378.37662 ]
 [549.2476   107.13793  800.       304.4565  ]
 [  0.       477.658    560.44324  800.      ]
 [  0.       331.54358  316.27057  800.      ]
 [280.2108     0.       800.       288.37537 ]
 [626.7549   190.80038  800.       512.5001  ]
 [404.15182  566.2378   800.       800.      ]
 [411.33533  459.94617  800.       800.      ]
 [328.70102  393.9956   800.       739.9203  ]
 [413.1931   259.71597  800.       573.63965 ]
 [515.89557  196.1736   800.       387.8381  ]
 [452.87314  504.30508  800.       701.6259  ]
 [  0.         0.       128.01971  603.8118  ]
 [591.97296  428.42703  800.       800.      ]
 [  0.         0.       406.56635  361.80563 ]
 [586.8722    82.020905 800.       486.02737 ]
 [524.27826  387.7734   800.       641.638   ]
 [  0.       367.66156  366.61368  800.      ]
 [  0.         0.       267.7129   539.1795  ]
 [472.73102  316.417    784.5113   800.      ]
 [410.28165  366.14935  731.1098   800.      ]
 [306.3931   

[[552.5362    102.56786   800.        300.51492  ]
 [  0.        331.63675   315.79288   800.       ]
 [ 80.55487   192.81297   581.59485   378.23547  ]
 [  0.        477.83228   558.8069    800.       ]
 [281.90955     0.        800.        289.7021   ]
 [405.52014   566.6483    800.        800.       ]
 [626.7911    188.10619   800.        512.155    ]
 [411.7147    459.5531    800.        800.       ]
 [  0.          0.        127.05328   584.14764  ]
 [324.2037    392.9885    800.        738.563    ]
 [407.17908   260.77237   800.        573.68665  ]
 [514.6443    194.8324    800.        384.24677  ]
 [  0.          0.        408.45203   363.1035   ]
 [456.0423    503.853     800.        700.3746   ]
 [591.72656   430.47247   800.        800.       ]
 [  0.          0.        269.0057    541.051    ]
 [525.32367   385.91092   800.        639.4374   ]
 [410.63293    65.878525  800.        405.71603  ]
 [470.97015   320.8447    782.01587   800.       ]
 [ 63.56697    72.9385    519.9

[[ 85.78048   189.27817   583.8855    378.5747   ]
 [551.0908    103.84774   800.        305.149    ]
 [  0.        320.23096   320.63446   800.       ]
 [  0.        477.46545   556.70276   800.       ]
 [279.02426     0.        800.        289.76154  ]
 [411.059     458.71667   800.        800.       ]
 [403.44333   565.95776   800.        800.       ]
 [624.7744    190.2638    800.        508.6411   ]
 [336.7882    389.44763   800.        741.6963   ]
 [409.385     262.18683   800.        570.01685  ]
 [469.633     310.92722   784.2964    800.       ]
 [534.76685   441.59564   800.        800.       ]
 [511.19427   194.90845   800.        384.36624  ]
 [  0.          0.        143.80026   575.7459   ]
 [  0.          0.        412.19846   357.50543  ]
 [242.1619    327.03516   671.1415    800.       ]
 [524.6021    384.01685   800.        641.17523  ]
 [  0.        365.5905    364.32855   800.       ]
 [589.3283     82.23672   800.        490.10272  ]
 [405.95245   360.91946   736.1

[[550.58276  102.44948  800.       305.897   ]
 [  0.       324.83743  321.6034   800.      ]
 [ 79.12686  192.0448   594.2181   377.92957 ]
 [  0.       481.8553   559.0605   800.      ]
 [410.2727   460.57614  800.       800.      ]
 [275.6685     0.       800.       291.3129  ]
 [400.5477   566.0143   800.       800.      ]
 [621.16113  186.8638   800.       508.15338 ]
 [511.57837  196.29492  800.       383.24118 ]
 [468.5523   306.1445   785.5464   800.      ]
 [340.7912   389.76813  800.       741.1282  ]
 [588.72864   77.24478  800.       489.08377 ]
 [  0.         0.       416.46585  358.03305 ]
 [299.86487  356.51807  598.301    663.6315  ]
 [407.88504  262.38593  800.       571.0276  ]
 [404.83722  359.05295  734.4723   800.      ]
 [244.52626  330.24774  664.01465  800.      ]
 [532.52264  442.81082  800.       800.      ]
 [  0.         0.       271.73776  565.36084 ]
 [  0.       364.83783  360.17633  800.      ]
 [152.76341   89.00774  661.3277   285.3565  ]
 [524.20996  

...

[[554.5307    105.515625  800.        305.3489   ]
 [  0.        342.20554   318.0798    800.       ]
 [441.7632    105.17354   800.        288.11853  ]
 [  0.        473.57544   544.1998    800.       ]
 [511.8412    200.96822   800.        383.66098  ]
 [407.56616   460.23474   800.        800.       ]
 [247.18454   338.92722   667.3831    800.       ]
 [397.77936   565.5783    800.        800.       ]
 [281.82977     0.        800.        291.8239   ]
 [464.77612   313.9245    777.5386    800.       ]
 [ 71.24722   191.72897   589.0306    380.94916  ]
 [411.96033   371.41455   725.69904   800.       ]
 [458.4693      0.        753.59375   159.07373  ]
 [619.8973    192.01782   800.        511.1467   ]
 [594.534     413.14764   800.        800.       ]
 [  0.          0.        411.18518   358.513    ]
 [345.2344     80.01705   796.68024   297.56613  ]
 [159.9911     91.18095   656.13635   290.42282  ]
 [344.1874    385.55164   800.        741.63586  ]
 [631.3959      8.265675  800. 

[[555.36835   103.39253   800.        305.13937  ]
 [  0.        356.34894   315.24286   800.       ]
 [509.04416   196.8427    800.        381.36823  ]
 [ 60.97944   190.70927   585.04675   379.80917  ]
 [439.90683   101.0766    800.        288.05084  ]
 [408.25516   462.03705   800.        800.       ]
 [361.48853   293.38947   541.224     644.72736  ]
 [397.61118   566.72046   800.        800.       ]
 [ 96.23568   100.87892   558.91187   269.67746  ]
 [  0.         88.99522   245.54686   294.23352  ]
 [411.07864   381.5196    720.755     800.       ]
 [  0.        470.46756   537.7059    800.       ]
 [328.5542     76.21651   792.97797   300.6236   ]
 [531.2189    459.42487   800.        800.       ]
 [413.0835     70.373825  800.        409.06262  ]
 [455.96246     0.        754.26685   158.82162  ]
 [630.23596     9.166849  800.        354.2532   ]
 [  0.          0.        407.9489    356.0392   ]
 [617.9513    190.84552   800.        509.93176  ]
 [202.14285   472.85794   652.3

[[5.5604126e+02 1.0268879e+02 8.0000000e+02 3.0539056e+02]
 [0.0000000e+00 3.1051456e+02 3.2808786e+02 8.0000000e+02]
 [0.0000000e+00 4.7238522e+02 5.4852740e+02 8.0000000e+02]
 [4.0310153e+02 4.5734586e+02 8.0000000e+02 8.0000000e+02]
 [5.9012225e+02 4.0338440e+02 8.0000000e+02 8.0000000e+02]
 [3.9109055e+02 5.6386304e+02 8.0000000e+02 8.0000000e+02]
 [4.0230341e+02 3.4439682e+02 7.3464587e+02 8.0000000e+02]
 [2.7502533e+02 0.0000000e+00 8.0000000e+02 2.9618219e+02]
 [2.4256839e+02 3.2528397e+02 6.8215314e+02 8.0000000e+02]
 [4.5181464e+02 2.9128891e+02 7.7723334e+02 8.0000000e+02]
 [4.4308862e+02 9.9016983e+01 8.0000000e+02 2.9056619e+02]
 [0.0000000e+00 5.9041119e+02 3.3921313e+02 8.0000000e+02]
 [6.1804865e+02 1.9446712e+02 8.0000000e+02 5.0913406e+02]
 [5.0781952e+02 1.9523306e+02 8.0000000e+02 3.8262125e+02]
 [6.9387558e+01 1.9301913e+02 5.9087164e+02 3.8242767e+02]
 [0.0000000e+00 0.0000000e+00 4.1650476e+02 3.6332089e+02]
 [0.0000000e+00 0.0000000e+00 2.6962152e+02 5.6896039e+0

[[  0.        307.60547   331.69968   800.       ]
 [553.56915   102.34032   800.        301.5731   ]
 [402.03165   458.63486   800.        800.       ]
 [405.25702   348.76578   737.08344   800.       ]
 [391.44107   564.88403   800.        800.       ]
 [  0.        478.40396   549.58325   800.       ]
 [276.7949      0.        800.        296.12317  ]
 [  0.          0.        269.56018   586.2413   ]
 [  0.          0.        414.63116   355.6275   ]
 [590.486     412.95032   800.        800.       ]
 [404.0858     67.7436    800.        409.39755  ]
 [352.37805   378.1345    800.        752.56836  ]
 [451.01346   291.68338   776.9559    800.       ]
 [246.98372   324.74557   678.7932    800.       ]
 [103.11212   101.34188   567.452     268.94812  ]
 [616.3432    193.92065   800.        512.3213   ]
 [507.5273    195.62851   800.        384.54355  ]
 [  0.        588.94916   345.0042    800.       ]
 [385.89514   273.4272    800.        557.61584  ]
 [624.7124      6.9485664 800. 

[[553.87555  104.23119  800.       302.91498 ]
 [  0.       322.48563  327.4721   800.      ]
 [404.2362   463.5418   800.       800.      ]
 [  0.       472.68802  536.68787  800.      ]
 [412.3759   362.65778  730.2923   800.      ]
 [400.2575    61.991573 800.       408.01993 ]
 [391.09518  567.1038   800.       800.      ]
 [281.22012    0.       800.       300.36038 ]
 [  0.         0.       418.9917   356.33115 ]
 [533.7191   444.50143  800.       800.      ]
 [417.786    290.43954  619.1491   671.0445  ]
 [348.94318  370.0872   800.       746.0491  ]
 [  0.         0.       267.07455  562.5655  ]
 [248.80579  344.3424   677.90985  800.      ]
 [499.14322  192.96675  800.       387.52112 ]
 [113.239815 525.3063   747.2171   800.      ]
 [354.12286  317.58176  539.6202   714.19885 ]
 [452.1293   304.79633  768.4007   800.      ]
 [630.07196   13.455009 800.       346.7307  ]
 [435.77518   99.73559  800.       288.45923 ]
 [582.3914    86.84974  800.       494.5421  ]
 [  0.       

In [ ]:
def multiply(x):
    return x * 800


import cv2
try:
    with detection_graph.as_default():
        with tf.Session() as sess:
                # Get handles to input and output tensors
                ops = tf.get_default_graph().get_operations()
                all_tensor_names = {output.name for op in ops for output in op.outputs}
                tensor_dict = {}
                for key in [
                  'num_detections', 'detection_boxes', 'detection_scores',
                  'detection_classes', 'detection_masks'
                ]:
                    tensor_name = key + ':0'
                    if tensor_name in all_tensor_names:
                        tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                      tensor_name)

                while True:
#                     ret, image_np = cap.read()
                    image_np = cv2.imread("crowd_2265.png")
                    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
                    image_np_expanded = np.expand_dims(image_np, axis=0)
                    # Actual detection.
                    output_dict = run_inference_for_single_image(image_np, detection_graph)
                    coordinates = np.asarray(output_dict['detection_boxes'])*800
                    print(np.asarray(coordinates))
                    
                    cv2.rectangle(image_np, (int(coordinates[-1][1]), int(coordinates[-1][2])), (int(coordinates[-1][3]), int(coordinates[-1][0])), (255,0,0), 2)
                    # Visualization of the results of a detection.
                    vis_util.visualize_boxes_and_labels_on_image_array(
                        image_np,
                        output_dict['detection_boxes'],
                        output_dict['detection_classes'],
                        output_dict['detection_scores'],
                        category_index,
                        instance_masks=output_dict.get('detection_masks'),
                        use_normalized_coordinates=True,
                        line_thickness=8)
                    cv2.imshow('object_detection', cv2.resize(image_np, (800, 800)))
                    if cv2.waitKey(25) & 0xFF == ord('q'):
                        cap.release()
                        cv2.destroyAllWindows()
                        break
except Exception as e:
    print(e)
    cap.release()

23.14769
